In [1]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
import os
os.getcwd()
#os.makedirs("/home/raghavendr48edu/logs")

#os.makedirs("/home/raghavendr48edu/logs")

# Configuration of the Spark Session
app_name = "instacart"
spark = SparkSession.builder \
       .appName(app_name)\
       .config("spark.ui.enabled", "true")\
       .config("spark.eventLog.enabled", "true")\
       .config("spark.eventLog.dir", "/home/raghavendr48edu/logs")\
       .config("spark.yarn.resourcemanager.hostname", "ip-10-1-1-204.ap-south-1.compute.internal") \
       .config("spark.yarn.resourcemanager.webapp.address", "ip-10-1-1-204.ap-south-1.compute.internal:6066") \
       .config("spark.executor.memory", "4g") \
       .config("spark.driver.memory", "4g") \
       .getOrCreate()

'''# Check the spark.eventLog.dir property
event_log_dir = spark.conf.get("spark.eventLog.dir")
print("Spark Event Log Directory:", event_log_dir)
print(spark.conf.get("spark.yarn.resourcemanager.hostname"))
print(spark.conf.get("spark.yarn.resourcemanager.webapp.address"))'''


'# Check the spark.eventLog.dir property\nevent_log_dir = spark.conf.get("spark.eventLog.dir")\nprint("Spark Event Log Directory:", event_log_dir)\nprint(spark.conf.get("spark.yarn.resourcemanager.hostname"))\nprint(spark.conf.get("spark.yarn.resourcemanager.webapp.address"))'

In [2]:
application_id = spark.sparkContext.applicationId
application_id

'local-1685259285241'

In [3]:
import psutil

# Get the current process ID (assuming it's the Spark driver process)
pid = os.getpid()

# Retrieve memory usage information
memory_info = psutil.Process(pid).memory_info()

# Print memory usage details
print(f"Memory Used: {memory_info.rss} bytes")
print(f"Memory Free: {psutil.virtual_memory().available} bytes")


Memory Used: 70336512 bytes
Memory Free: 162586284032 bytes


### 1. Load data into Spark DataFrame

In [4]:
orders = spark.read.csv("orders.csv",inferSchema=True,header=True)
products=spark.read.csv("products.csv",inferSchema=True,header=True)
departments=spark.read.csv("departments.csv",inferSchema=True,header=True)
aisles=spark.read.csv("aisles.csv",inferSchema=True,header=True)
order_products__prior=spark.read.csv("order_products__prior.csv",inferSchema=True,header=True)
order_products__train=spark.read.csv("order_products__train.csv",inferSchema=True,header=True)

orders.createOrReplaceTempView("orders")
products.createOrReplaceTempView("products")
departments.createOrReplaceTempView("departments")
aisles.createOrReplaceTempView("aisles")
order_products__prior.createOrReplaceTempView("order_products__prior")
order_products__train.createOrReplaceTempView("order_products__train")


orders.show(10)
products.show(10)
departments.show(10)
aisles.show(10)
order_products__prior.show(10)
order_products__train.show(10)

+--------+-------+--------+------------+---------+-----------------+----------------------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
| 2539329|      1|   prior|           1|        2|                8|                  null|
| 2398795|      1|   prior|           2|        3|                7|                  15.0|
|  473747|      1|   prior|           3|        3|               12|                  21.0|
| 2254736|      1|   prior|           4|        4|                7|                  29.0|
|  431534|      1|   prior|           5|        4|               15|                  28.0|
| 3367565|      1|   prior|           6|        2|                7|                  19.0|
|  550135|      1|   prior|           7|        1|                9|                  20.0|
| 3108588|      1|   prior|           8|        1|               14|            

In [5]:
orders.count(),products.count(),departments.count(),aisles.count(),order_products__prior.count(),order_products__train.count(),

(3421083, 49688, 21, 134, 32434489, 1384617)

In [6]:
spark.sql("select count(distinct order_id) from order_products__prior").show() #3214874
spark.sql("select count(distinct order_id) from order_products__train").show() #131209

#3421083-(3214874+131209)
#75000 records doesn't have product_id

+------------------------+
|count(DISTINCT order_id)|
+------------------------+
|                 3214874|
+------------------------+

+------------------------+
|count(DISTINCT order_id)|
+------------------------+
|                  131209|
+------------------------+



### Checking null counts for each column in each dataframe

In [7]:
df_names = ['orders', 'products', 'departments', 'aisles', 'order_products__prior', 'order_products__train']
null_counts = [
    [(col, df.where(df[col].isNull()).count()) for col in df.columns]
    for df_name in df_names
    for df in [globals()[df_name]]
]
'''
# Print the results
for i, df_name in enumerate(df_names):
    print(f"Null counts in DataFrame '{df_name}':")
    for col, count in null_counts[i]:
        print(f"{col}: {count}")
    print()
    '''
null_counts

[[('order_id', 0),
  ('user_id', 0),
  ('eval_set', 0),
  ('order_number', 0),
  ('order_dow', 0),
  ('order_hour_of_day', 0),
  ('days_since_prior_order', 206209)],
 [('product_id', 0),
  ('product_name', 0),
  ('aisle_id', 0),
  ('department_id', 0)],
 [('department_id', 0), ('department', 0)],
 [('aisle_id', 0), ('aisle', 0)],
 [('order_id', 0),
  ('product_id', 0),
  ('add_to_cart_order', 0),
  ('reordered', 0)],
 [('order_id', 0),
  ('product_id', 0),
  ('add_to_cart_order', 0),
  ('reordered', 0)]]

### 2. Merge all the data frames based on the common key and create a single DataFrame and 3. check missing data

In [8]:
df_merge = aisles.join(products, "aisle_id","outer").join(departments, "department_id","outer")
df_merge.show(5)

+-------------+--------+------------+----------+--------------------+------------+
|department_id|aisle_id|       aisle|product_id|        product_name|  department|
+-------------+--------+------------+----------+--------------------+------------+
|           12|     122|meat counter|       175|        T Bone Steak|meat seafood|
|           12|     122|meat counter|       194|      Lamb Rib Chops|meat seafood|
|           12|     122|meat counter|       692|         Skirt Steak|meat seafood|
|           12|     122|meat counter|      1938|      Beef Stew Meat|meat seafood|
|           12|     122|meat counter|      2301|Beef Loin Porterh...|meat seafood|
+-------------+--------+------------+----------+--------------------+------------+
only showing top 5 rows



In [9]:
df_merge.count()

49688

In [10]:
from pyspark.sql.functions import col, isnull, isnan, sum

null_counts = df_merge.select([sum(col(column).isNull().cast("int")).alias(column) for column in df_merge.columns])
null_counts.show()


+-------------+--------+-----+----------+------------+----------+
|department_id|aisle_id|aisle|product_id|product_name|department|
+-------------+--------+-----+----------+------------+----------+
|            0|       0|    1|         0|           0|         1|
+-------------+--------+-----+----------+------------+----------+



In [11]:

from functools import reduce


# Create a list comprehension to generate the condition for each column
null_conditions = [col(column).isNull() for column in df_merge.columns]
# Apply the conditions using the OR operator to filter the dataframe
null_records = df_merge.filter(reduce(lambda a, b: a | b, null_conditions))
# Display the null records
null_records.show()


+-------------+--------+-----+----------+--------------------+----------+
|department_id|aisle_id|aisle|product_id|        product_name|department|
+-------------+--------+-----+----------+--------------------+----------+
|         Red"| Blunted| null|      6816|"Scotch Kids 5"" ...|      null|
+-------------+--------+-----+----------+--------------------+----------+



In [12]:
products[products['product_id']==6816].show()

+----------+--------------------+--------+-------------+
|product_id|        product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
|      6816|"Scotch Kids 5"" ...| Blunted|         Red"|
+----------+--------------------+--------+-------------+



In [13]:
order_products__prior[order_products__prior['product_id']==6816].show()

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|  482604|      6816|                8|        0|
| 1776850|      6816|                3|        0|
| 2662328|      6816|                8|        0|
+--------+----------+-----------------+---------+



In [14]:
order_products__train[order_products__train['product_id']==6816].show()

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
+--------+----------+-----------------+---------+




* For 3 records product_id=6816 aisle_id,department_id is not there but there are 3 records in order.
* let us keep in mind and drop if necessary

In [15]:
order_products__prior[(order_products__prior["order_id"]>0) & (order_products__prior["order_id"]<10)].select("order_id").distinct().show()
order_products__train[(order_products__train["order_id"]>0) & (order_products__train["order_id"]<10)].select("order_id").distinct().show()

+--------+
|order_id|
+--------+
|       6|
|       3|
|       5|
|       9|
|       4|
|       8|
|       7|
|       2|
+--------+

+--------+
|order_id|
+--------+
|       1|
+--------+



In [16]:
df_order_products=order_products__prior.union(order_products__train)
#df_order_products.select("order_id").distinct().count()
df_order_products.agg(F.countDistinct("order_id").alias("distinct_count")).show()



+--------------+
|distinct_count|
+--------------+
|       3346083|
+--------------+



In [17]:
df_order_products.count()

33819106

#### limit is used to control too many open files error

In [19]:
df_orders_outer=orders.limit(350000000).join(df_order_products.limit(350000000),"order_id","outer")
df_orders_outer.count()

33894106

In [20]:
null_counts = df_orders_outer.select([sum(col(column).isNull().cast("int")).alias(column) for column in df_orders_outer.columns])
# Display the null counts
null_counts.show()


+--------+-------+--------+------------+---------+-----------------+----------------------+----------+-----------------+---------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|product_id|add_to_cart_order|reordered|
+--------+-------+--------+------------+---------+-----------------+----------------------+----------+-----------------+---------+
|       0|      0|       0|           0|        0|                0|               2078068|     75000|            75000|    75000|
+--------+-------+--------+------------+---------+-----------------+----------------------+----------+-----------------+---------+



#### 75000 orders has no details of productId,add_to_cart_order, reorder

In [21]:
df=df_orders_outer.join(df_merge,'product_id','outer')
df.show(5)

+----------+--------+-------+--------+------------+---------+-----------------+----------------------+-----------------+---------+-------------+--------+------------+------------+----------+
|product_id|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|add_to_cart_order|reordered|department_id|aisle_id|       aisle|product_name|department|
+----------+--------+-------+--------+------------+---------+-----------------+----------------------+-----------------+---------+-------------+--------+------------+------------+----------+
|       148|     224| 109534|   prior|           6|        0|               14|                  13.0|                2|        1|            4|      24|fresh fruits|  Nectarines|   produce|
|       148|    1452|  72231|   prior|          16|        2|               22|                  15.0|                5|        1|            4|      24|fresh fruits|  Nectarines|   produce|
|       148|    2688|  99750|   prior|       

In [22]:
df.count()

33894109

In [23]:
df[df['product_id']==6816].show()

+----------+--------+-------+--------+------------+---------+-----------------+----------------------+-----------------+---------+-------------+--------+-----+--------------------+----------+
|product_id|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|add_to_cart_order|reordered|department_id|aisle_id|aisle|        product_name|department|
+----------+--------+-------+--------+------------+---------+-----------------+----------------------+-----------------+---------+-------------+--------+-----+--------------------+----------+
|      6816|  482604|  16340|   prior|          51|        4|               11|                   2.0|                8|        0|         Red"| Blunted| null|"Scotch Kids 5"" ...|      null|
|      6816| 1776850| 120158|   prior|          14|        4|               22|                   6.0|                3|        0|         Red"| Blunted| null|"Scotch Kids 5"" ...|      null|
|      6816| 2662328|  72754|   prior|  

* above are 3 extra records. Total 75003 null records due to outer join  except days_since_prior_order column

### 4.List the most ordered products (top 10)

In [24]:
df.groupby('product_id','product_name').count().orderBy(col('count').desc()).limit(10).show()

+----------+--------------------+------+
|product_id|        product_name| count|
+----------+--------------------+------+
|     24852|              Banana|491291|
|     13176|Bag of Organic Ba...|394930|
|     21137|Organic Strawberries|275577|
|     21903|Organic Baby Spinach|251705|
|     47209|Organic Hass Avocado|220877|
|     47766|     Organic Avocado|184224|
|     47626|         Large Lemon|160792|
|     16797|        Strawberries|149445|
|     26209|               Limes|146660|
|     27845|  Organic Whole Milk|142813|
+----------+--------------------+------+



In [25]:
from pyspark.sql.functions import col, rank, count,dense_rank
from pyspark.sql.window import Window

df.withColumn('count', count('product_id').over(Window.partitionBy('product_id'))) \
                 .withColumn('rank', dense_rank().over(Window.orderBy(col('count').desc()))) \
                 .select('product_id', 'product_name', 'count','rank').distinct().show()


+----------+--------------------+------+----+
|product_id|        product_name| count|rank|
+----------+--------------------+------+----+
|     24852|              Banana|491291|   1|
|     13176|Bag of Organic Ba...|394930|   2|
|     21137|Organic Strawberries|275577|   3|
|     21903|Organic Baby Spinach|251705|   4|
|     47209|Organic Hass Avocado|220877|   5|
|     47766|     Organic Avocado|184224|   6|
|     47626|         Large Lemon|160792|   7|
|     16797|        Strawberries|149445|   8|
|     26209|               Limes|146660|   9|
|     27845|  Organic Whole Milk|142813|  10|
|     27966| Organic Raspberries|142603|  11|
|     22935|Organic Yellow Onion|117716|  12|
|     24964|      Organic Garlic|113936|  13|
|     45007|    Organic Zucchini|109412|  14|
|     39275| Organic Blueberries|105026|  15|
|     49683|      Cucumber Kirby| 99728|  16|
|     28204|  Organic Fuji Apple| 92889|  17|
|      5876|       Organic Lemon| 91251|  18|
|     40706|Organic Grape Tom...| 

In [26]:
df.createOrReplaceTempView("Instacart")

In [27]:
spark.sql("select distinct product_id,product_name,count(product_id) OVER(partition by product_id) as count from Instacart order by count desc").show(10)
#spark.sql("select distinct product_id,product_name,count(product_id) OVER(partition by product_id order by count(product_id) desc) as count from Instacart ").show(10)

+----------+--------------------+------+
|product_id|        product_name| count|
+----------+--------------------+------+
|     24852|              Banana|491291|
|     13176|Bag of Organic Ba...|394930|
|     21137|Organic Strawberries|275577|
|     21903|Organic Baby Spinach|251705|
|     47209|Organic Hass Avocado|220877|
|     47766|     Organic Avocado|184224|
|     47626|         Large Lemon|160792|
|     16797|        Strawberries|149445|
|     26209|               Limes|146660|
|     27845|  Organic Whole Milk|142813|
+----------+--------------------+------+
only showing top 10 rows



### 5.Do people usually reorder the same previous ordered products?

In [28]:
windowSpec = Window.partitionBy("product_id")
df.withColumn("count", F.count("product_id").over(windowSpec)) \
           .withColumn("sum_reordered", F.sum("reordered").over(windowSpec)) \
           .select("product_id", "count", "sum_reordered") \
           .distinct().withColumn("reordering_probability",F.col("sum_reordered")/F.col("count")) \
            .orderBy(F.desc("reordering_probability")).show()

+----------+-----+-------------+----------------------+
|product_id|count|sum_reordered|reordering_probability|
+----------+-----+-------------+----------------------+
|      6433|   69|           65|    0.9420289855072463|
|      2075|   90|           84|    0.9333333333333333|
|     43553|   13|           12|    0.9230769230769231|
|     27740|  102|           94|    0.9215686274509803|
|     14609|   35|           32|    0.9142857142857143|
|     13875|   45|           41|    0.9111111111111111|
|     39992|   22|           20|    0.9090909090909091|
|      5868|   30|           27|                   0.9|
|     36543|   69|           62|    0.8985507246376812|
|     26093|   67|           60|    0.8955223880597015|
|      4212|   38|           34|    0.8947368421052632|
|     35604|  104|           93|    0.8942307692307693|
|     38438|   28|           25|    0.8928571428571429|
|     38251|  111|           99|    0.8918918918918919|
|     36801|   99|           88|    0.8888888888

In [29]:
df.groupBy("product_id") \
           .agg(F.count("product_id").alias("count"), \
                F.sum("reordered").alias("sum_reordered")) \
            .withColumn("reordering_probability",F.col("sum_reordered")/F.col("count")) \
            .orderBy(F.desc("reordering_probability")).show()

+----------+-----+-------------+----------------------+
|product_id|count|sum_reordered|reordering_probability|
+----------+-----+-------------+----------------------+
|      6433|   69|           65|    0.9420289855072463|
|      2075|   90|           84|    0.9333333333333333|
|     43553|   13|           12|    0.9230769230769231|
|     27740|  102|           94|    0.9215686274509803|
|     14609|   35|           32|    0.9142857142857143|
|     13875|   45|           41|    0.9111111111111111|
|     39992|   22|           20|    0.9090909090909091|
|      5868|   30|           27|                   0.9|
|     36543|   69|           62|    0.8985507246376812|
|     26093|   67|           60|    0.8955223880597015|
|      4212|   38|           34|    0.8947368421052632|
|     35604|  104|           93|    0.8942307692307693|
|     38438|   28|           25|    0.8928571428571429|
|     38251|  111|           99|    0.8918918918918919|
|     36801|   99|           88|    0.8888888888

In [30]:
spark.sql("select product_id,count(product_id),sum(reordered),sum(reordered)/count(product_id) as reordering_probability from Instacart\
          group by product_id order by reordering_probability desc").show(10)

+----------+-----------------+--------------+----------------------+
|product_id|count(product_id)|sum(reordered)|reordering_probability|
+----------+-----------------+--------------+----------------------+
|      6433|               69|            65|    0.9420289855072463|
|      2075|               90|            84|    0.9333333333333333|
|     43553|               13|            12|    0.9230769230769231|
|     27740|              102|            94|    0.9215686274509803|
|     14609|               35|            32|    0.9142857142857143|
|     13875|               45|            41|    0.9111111111111111|
|     39992|               22|            20|    0.9090909090909091|
|      5868|               30|            27|                   0.9|
|     36543|               69|            62|    0.8985507246376812|
|     26093|               67|            60|    0.8955223880597015|
+----------+-----------------+--------------+----------------------+
only showing top 10 rows



In [31]:
spark.sql("SELECT distinct product_id, COUNT(product_id) OVER (PARTITION BY product_id) AS count, \
                   SUM(reordered) OVER (PARTITION BY product_id) AS sum_reordered, \
                   SUM(reordered) OVER (PARTITION BY product_id) / COUNT(product_id) OVER (PARTITION BY product_id) AS reordering_probability \
                   FROM Instacart \
                   ORDER BY reordering_probability DESC \
                   LIMIT 10").show()


+----------+-----+-------------+----------------------+
|product_id|count|sum_reordered|reordering_probability|
+----------+-----+-------------+----------------------+
|      6433|   69|           65|    0.9420289855072463|
|      2075|   90|           84|    0.9333333333333333|
|     43553|   13|           12|    0.9230769230769231|
|     27740|  102|           94|    0.9215686274509803|
|     14609|   35|           32|    0.9142857142857143|
|     13875|   45|           41|    0.9111111111111111|
|     39992|   22|           20|    0.9090909090909091|
|      5868|   30|           27|                   0.9|
|     36543|   69|           62|    0.8985507246376812|
|     26093|   67|           60|    0.8955223880597015|
+----------+-----+-------------+----------------------+



In [32]:
#Overall mean of the reordering probability will decide the overall attitude of public in reordering

spark.sql('Select avg(a.reordering_probability) as reordering_sentiment from \
          (Select product_id ,count(product_id) as purchase_count , sum(reordered) as reorderd_count , \
          sum(reordered)/count(product_id) as reordering_probability  from order_products__prior \
          group by product_id  order by reordering_probability desc) as a').show()


+--------------------+
|reordering_sentiment|
+--------------------+
|  0.3664610156232542|
+--------------------+



### 6.List most reordered products

In [33]:
spark.sql("select product_id,product_name,sum(reordered) from Instacart group by product_id,product_name order by sum(reordered) desc").show(10)

+----------+--------------------+--------------+
|product_id|        product_name|sum(reordered)|
+----------+--------------------+--------------+
|     24852|              Banana|        415166|
|     13176|Bag of Organic Ba...|        329275|
|     21137|Organic Strawberries|        214448|
|     21903|Organic Baby Spinach|        194939|
|     47209|Organic Hass Avocado|        176173|
|     47766|     Organic Avocado|        140270|
|     27845|  Organic Whole Milk|        118684|
|     47626|         Large Lemon|        112178|
|     27966| Organic Raspberries|        109688|
|     16797|        Strawberries|        104588|
+----------+--------------------+--------------+
only showing top 10 rows



In [34]:
df.groupBy('product_id').agg(F.sum('reordered').alias("sum_reordered")).orderBy(F.desc('sum_reordered')).show(10)

+----------+-------------+
|product_id|sum_reordered|
+----------+-------------+
|     24852|       415166|
|     13176|       329275|
|     21137|       214448|
|     21903|       194939|
|     47209|       176173|
|     47766|       140270|
|     27845|       118684|
|     47626|       112178|
|     27966|       109688|
|     16797|       104588|
+----------+-------------+
only showing top 10 rows



### 7.Most important department and aisle (by number of products) and 8.Get the Top 10 departments

In [35]:
spark.sql("Select department_id,count(product_id)  from products group by department_id  order by count(product_id)  desc limit 10").show(10)

+-------------+-----------------+
|department_id|count(product_id)|
+-------------+-----------------+
|           11|             6563|
|           19|             6264|
|           13|             5371|
|            7|             4365|
|            1|             4007|
|           16|             3449|
|           17|             3084|
|           15|             2092|
|            9|             1858|
|            4|             1684|
+-------------+-----------------+



In [36]:
spark.sql("SELECT distinct department_id, count(product_id) over (PARTITION BY department_id) as count_product FROM products \
            order by count_product desc limit 10").show()

+-------------+-------------+
|department_id|count_product|
+-------------+-------------+
|           11|         6563|
|           19|         6264|
|           13|         5371|
|            7|         4365|
|            1|         4007|
|           16|         3449|
|           17|         3084|
|           15|         2092|
|            9|         1858|
|            4|         1684|
+-------------+-------------+



In [37]:
spark.sql("SELECT p.department_id, d.department, count_product \
           FROM (SELECT department_id, COUNT(product_id) AS count_product \
                 FROM products \
                 GROUP BY department_id) p \
           JOIN departments d ON p.department_id = d.department_id \
           ORDER BY count_product DESC \
           LIMIT 10").show()


+-------------+---------------+-------------+
|department_id|     department|count_product|
+-------------+---------------+-------------+
|           11|  personal care|         6563|
|           19|         snacks|         6264|
|           13|         pantry|         5371|
|            7|      beverages|         4365|
|            1|         frozen|         4007|
|           16|     dairy eggs|         3449|
|           17|      household|         3084|
|           15|   canned goods|         2092|
|            9|dry goods pasta|         1858|
|            4|        produce|         1684|
+-------------+---------------+-------------+



In [38]:
spark.sql("SELECT p.department_id, d.department, COUNT(p.product_id) AS count_product \
           FROM products p \
           JOIN departments d ON p.department_id = d.department_id \
           GROUP BY p.department_id, d.department \
           ORDER BY count_product DESC \
           LIMIT 10").show()


+-------------+---------------+-------------+
|department_id|     department|count_product|
+-------------+---------------+-------------+
|           11|  personal care|         6563|
|           19|         snacks|         6264|
|           13|         pantry|         5371|
|            7|      beverages|         4365|
|            1|         frozen|         4007|
|           16|     dairy eggs|         3449|
|           17|      household|         3084|
|           15|   canned goods|         2092|
|            9|dry goods pasta|         1858|
|            4|        produce|         1684|
+-------------+---------------+-------------+



In [39]:
df.select('department_id','department','product_id').groupby('department_id','department').agg(F.countDistinct('product_id').alias('count_product')).orderBy(F.desc('count_product')).show(100)

+-------------+---------------+-------------+
|department_id|     department|count_product|
+-------------+---------------+-------------+
|           11|  personal care|         6563|
|           19|         snacks|         6264|
|           13|         pantry|         5371|
|            7|      beverages|         4365|
|            1|         frozen|         4007|
|           16|     dairy eggs|         3449|
|           17|      household|         3084|
|           15|   canned goods|         2092|
|            9|dry goods pasta|         1858|
|            4|        produce|         1684|
|            3|         bakery|         1516|
|           20|           deli|         1322|
|           21|        missing|         1258|
|            6|  international|         1139|
|           14|      breakfast|         1115|
|           18|         babies|         1081|
|            5|        alcohol|         1054|
|            8|           pets|          972|
|           12|   meat seafood|   

### 9.List top 10 products ordered in the morning (6 AM to 11 AM)

In [40]:
df.filter((col('order_hour_of_day')>=6) & (col('order_hour_of_day')<=11)).groupby('product_id','product_name').count().orderBy(col('count').desc()).limit(10).show()

+----------+--------------------+------+
|product_id|        product_name| count|
+----------+--------------------+------+
|     24852|              Banana|169965|
|     13176|Bag of Organic Ba...|135417|
|     21137|Organic Strawberries| 92499|
|     21903|Organic Baby Spinach| 82578|
|     47209|Organic Hass Avocado| 72545|
|     47766|     Organic Avocado| 59603|
|     47626|         Large Lemon| 53479|
|     16797|        Strawberries| 52155|
|     27966| Organic Raspberries| 49751|
|     27845|  Organic Whole Milk| 49747|
+----------+--------------------+------+

